In [ ]:
!nvidia-smi

Fri Jan 16 05:14:00 2026       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.15              Driver Version: 550.54.15      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  Tesla T4                       Off |   00000000:00:04.0 Off |                    0 |
| N/A   41C    P8             11W /   70W |       0MiB /  15360MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [ ]:
!pip install -q transformers accelerate sentencepiece torch tqdm pandas

In [ ]:
import torch
import json
import pandas as pd
from tqdm import tqdm
from transformers import AutoTokenizer, AutoModelForCausalLM

In [ ]:
!wget -q http://curtis.ml.cmu.edu/datasets/hotpot/hotpot_train_v1.1.json

In [ ]:
!ls

hotpot_train_v1.1.json	sample_data


In [ ]:
with open("hotpot_train_v1.1.json", "r") as f:
    data = json.load(f)

print("Total questions:", len(data))

Total questions: 90447


In [ ]:
data = data[:200]   # DO NOT increase yet
print("Using questions:", len(data))

Using questions: 200


In [ ]:
df = pd.DataFrame({
    "qid": [x["_id"] for x in data],
    "question": [x["question"] for x in data],
    "gold_answer": [x["answer"] for x in data]
})

df.head()

,qid,question,gold_answer
0,5a7a06935542990198eaf050,Which magazine was started first Arthur's Maga...,Arthur's Magazine
1,5a879ab05542996e4f30887e,The Oberoi family is part of a hotel company t...,Delhi
2,5a8d7341554299441c6b9fe5,Musician and satirist Allie Goertz wrote a son...,President Richard Nixon
3,5a82171f5542990a1d231f4a,What nationality was James Henry Miller's wife?,American
4,5a84dd955542997b5ce3ff79,Cadmium Chloride is slightly soluble in this c...,alcohol


In [ ]:
model_name = "mistralai/Mistral-7B-Instruct-v0.2"

In [ ]:
tokenizer = AutoTokenizer.from_pretrained(model_name)

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json: 0.00B [00:00, ?B/s]

tokenizer.model:   0%|          | 0.00/493k [00:00<?, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/414 [00:00<?, ?B/s]

In [ ]:
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    torch_dtype=torch.float16,
    device_map="auto"
)


config.json:   0%|          | 0.00/596 [00:00<?, ?B/s]

`torch_dtype` is deprecated! Use `dtype` instead!


model.safetensors.index.json: 0.00B [00:00, ?B/s]

Fetching 3 files:   0%|          | 0/3 [00:00<?, ?it/s]

model-00002-of-00003.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

model-00003-of-00003.safetensors:   0%|          | 0.00/4.54G [00:00<?, ?B/s]

model-00001-of-00003.safetensors:   0%|          | 0.00/4.94G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/111 [00:00<?, ?B/s]

In [ ]:
def query_mistral(prompt, max_tokens=128):
    inputs = tokenizer(prompt, return_tensors="pt").to(model.device)
    outputs = model.generate(
        **inputs,
        max_new_tokens=max_tokens,
        do_sample=False
    )
    return tokenizer.decode(outputs[0], skip_special_tokens=True)

In [ ]:
def baseline_prompt(question):
    return f"""Answer the following multi-hop question step by step and provide a final answer.

Question: {question}
"""

In [ ]:
def sufficiency_prompt(question):
    return f"""Reason step by step.
At each step, check whether the available evidence is sufficient.
If evidence is insufficient or contradictory, respond with:
INSUFFICIENT INFORMATION.
Do not guess.

Question: {question}
"""

In [ ]:
q = df.iloc[0]["question"]

print("QUESTION:", q)

print("\nBASELINE:")
print(query_mistral(baseline_prompt(q)))

print("\nOURS:")
print(query_mistral(sufficiency_prompt(q)))

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


QUESTION: Which magazine was started first Arthur's Magazine or First for Women?

BASELINE:


Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


Answer the following multi-hop question step by step and provide a final answer.

Question: Which magazine was started first Arthur's Magazine or First for Women?

Step 1: Find the start date for Arthur's Magazine.
- Go to the Arthur's Magazine website: https://arthurmag.com/
- Click on "About Us" in the footer menu.
- Scroll down to the "History" section.
- The first issue of Arthur's Magazine was published in 1997.

Step 2: Find the start date for First for Women.
- Go to the First for Women website: https://www.firstforwomen.com/
- Click on "About Us" in the footer menu

OURS:
Reason step by step.
At each step, check whether the available evidence is sufficient.
If evidence is insufficient or contradictory, respond with:
INSUFFICIENT INFORMATION.
Do not guess.

Question: Which magazine was started first Arthur's Magazine or First for Women?
Answer: INSUFFICIENT INFORMATION.

There is no publicly available information that definitively states which of these magazines was started firs

In [ ]:
baseline_outputs = []

for q in tqdm(df["question"]):
    baseline_outputs.append(query_mistral(baseline_prompt(q)))

df["baseline_output"] = baseline_outputs

100%|██████████| 200/200 [3:39:56<00:00, 65.98s/it]


In [ ]:
ours_outputs = []

for q in tqdm(df["question"]):
    ours_outputs.append(query_mistral(sufficiency_prompt(q)))

df["ours_output"] = ours_outputs

NameError: name 'tqdm' is not defined